In [ ]:
import pandas as pd
import json
import re
import os
import numpy as np
#from tinydb import TinyDB
#import requests
#from bs4 import BeautifulSoup
#from urllib.request import urlopen

# NLP
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords

workDir = '/home/lefko/git/ut-health-project'
dbfile = workDir + '/db/tweets.csv'

In [ ]:
# check if the JSON file can be accessed
try:
    f = open(dbfile)
except IOError:
    print("File not accessible")
finally:
    f.close()


In [ ]:
def getCitiesPerState(state):
    '''
        @param state must begin with an uppercase letter
        scrape the list of cities and towns in the respective state
    '''
    raise DeprecationWarning('Not working, sadly.')
    cities = []
    url = 'https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_' + state
    print('url to scrape:', url)
    website = requests.get(url)
    # load the retrieved site with BS
    soup = BeautifulSoup(website.text,'html.parser')
    
    # cities/towns are stored in an element with class 'wikitable sortable'
    cities_table = soup.find_all("table", class_ = "wikitable sortable")
    print(len(cities_table))
    '''
    cities_links = cities_table.findAll('a') # city names are links in the table
    for link in cities_links:
        cities.append(link.get('title'))
    '''
    return cities

def readCitiesFromFile(state):
    cities = []
    with open(workDir + '/data/cities_' + state + '.txt') as txt:
        for line in txt:
            cities.append(line.strip().lower())

    return pd.DataFrame(cities, columns = ['city'])

def readKeywords():
    keywords = []

    with open(workDir + '/data/keywords.txt') as txt:
        for line in txt:
            keywords.append(line.strip().lower())

    print(keywords)
    return keywords


In [ ]:
cities_colorado = readCitiesFromFile('co')
cities_california = readCitiesFromFile('ca')
cities_massachussetts = readCitiesFromFile('ma')
cities_all = cities_california + cities_colorado + cities_massachussetts
print(cities_colorado[0:10])
print(cities_california[0:10])
print(cities_massachussetts[0:10])

In [ ]:
tweets = pd.read_csv(dbfile, names=['tweet_id', 'created', 'loc', 'text'], header=None)
tweets = tweets[tweets['loc'].isnull() == False] # sort out NaN places
tweets = tweets[tweets['loc'].str.contains('\d') == False]
tweets = tweets.applymap(lambda s:s.lower() if type(s) == str else s)
print(len(tweets), 'tweets loaded!')

In [ ]:
'''
    Tweets are saved in a JSON. Read this file and convert it's contents to a pandas dataframe.

def accessDB():
    db = TinyDB(dbfile)
    return db.all()


tweets = accessDB()
df = pd.DataFrame()

for i, tweet in enumerate(tweets):
    df = df.append(pd.DataFrame(tweet, index = [i]))

print(len(df), 'tweets loaded!')
'''


In [ ]:
print(tweets.head(15))
print(tweets.tail(15))

In [ ]:
# How unique are the users locations?
print(tweets['loc'].unique())
#print(tweets['loc'].unique().tolist())

In [ ]:
# check for some real location names
tweets[tweets['loc'].isin(cities_all) == True]

In [ ]:
# These tweets contain locations, either real or fake
tweets[tweets['loc'].str.contains('|'.join([' ma', ' co', ' ca']))]

In [ ]:
# average tweet length
np.mean(tweets['text'].str.len())

In [ ]:
# tweet tokenization and stopword removal
_stopwords = ['https', 'co', 'ct'] + stopwords.words('english')
_keywords = readKeywords()
def processTweet(tweet):
    lem = WordNetLemmatizer()

    tweet = [word for word in word_tokenize(tweet) if word.isalpha() == True]
    tweet = [lem.lemmatize(word,'v') for word in tweet]
    return [word for word in tweet if word not in _stopwords]

'''
A vocabulary in Natural Language Processing is a list of all speech segments available for the model. In our case, this includes all the words resident in the Training set we have, as the model can make use of all of them relatively equally — at this point, to say the least
'''
def buildVocab(tokenizedData):
    wordlist = nltk.FreqDist(tokenizedData)
    #wordlist.plot(50, cumulative = True)
    word_features = wordlist.keys()
    
    return word_features

def filterKeywords(tweet):
    if any(x in processTweet(tweet) for x in _keywords):
        print(tweet)
    #if(tweet in _keywords):
    #    print(tweet)

processed_tweets = []
for tweet in tweets['text']:
    filterKeywords(tweet)
    #tweet_tokens = processTweet(tweet)
    #for token in tweet_tokens:
    #    processed_tweets.append(token)

#print(processed_tweets[0:10])
#print(buildVocab(processed_tweets))
#buildVocab(processed_tweets[:50])
